In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

In [4]:
def rsin():
    rand = np.random.random(size=5)
    x = np.linspace(-1, 1, 32)
    return (np.sin(x*(2+rand[0])+rand[1]*3)+rand[2])*rand[3]*3 * ((round(rand[4])*-2)+1)
def rquad(): #farquad
    rand = np.random.random(size=5)
    x = np.linspace(-1, 1, 32)
    return ((x+(((round(rand[0])*-2)+1) * rand[1]))**2)+(rand[2] * 3 *rand[3]) * ((round(rand[4])*-2)+1)

def fsin(f):
    rand = np.random.random(size=5)
    x = np.linspace(-1, 1, 512)
    y = np.sin((x+rand[0]) * f) * rand[1] + rand[2]
    return y
    

In [5]:
class sorq(Dataset):
    def __init__(
            self,
            length=1000,
            device="cuda:0"
    ):
        self.length = length
        self.device = device

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        output = rsin() if index%2 == 1 else rquad()
        return (torch.tensor(output).view(-1,1).to(self.device).to(torch.float),
                torch.full((len(output),),index%2).view(-1,1).to(self.device).to(torch.float))

    def __str__(self) -> str:
        return f"{self.length}"

dataset = sorq(length=1_0_000)

In [6]:
# c,d = next(iter(dataloader))
# for i,j in zip(c,d):
#     print(j.T,i.shape,j.shape)
#     plt.plot(i.detach().cpu())
#     plt.show()

In [7]:
class LSTMDiscriminatorHC(nn.Module):
    """An LSTM based discriminator. It expects a sequence as input and outputs a probability for each element. 
    Args:
        in_dim: Input noise dimensionality
        n_layers: number of lstm layers
        hidden_dim: dimensionality of the hidden layer of lstms
    Inputs: sequence of shape (batch_size, seq_len, in_dim)
    Output: sequence of shape (batch_size, seq_len, 1)
    """

    def __init__(self, in_dim, n_layers=1, hidden_dim=256):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.linear_hidden = nn.Sequential(nn.Linear(hidden_dim*self.n_layers, 1), nn.Sigmoid())
        self.linear_cellst = nn.Sequential(nn.Linear(hidden_dim*self.n_layers, 1), nn.Sigmoid())
        self.linear_out = nn.Bilinear(1,1,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        batch_size, seq_len = input.size(0), input.size(1)
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")
        c_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")

        recurrent_features, (h_out,c_out) = self.lstm(input, (h_0, c_0))
        # outputs = self.linear_recurrent(recurrent_features.contiguous().view(batch_size*seq_len, self.hidden_dim))
        # outputs = outputs.view(batch_size, seq_len, 1)
        ph_out = self.linear_hidden(h_out.view(batch_size,self.hidden_dim*self.n_layers))
        pc_out = self.linear_cellst(c_out.view(batch_size,self.hidden_dim*self.n_layers))

        return self.sigmoid(self.linear_out(ph_out,pc_out))

In [8]:
class LSTMDiscriminatorHO(nn.Module):
    """An LSTM based discriminator. It expects a sequence as input and outputs a probability for each element. 
    Args:
        in_dim: Input noise dimensionality
        n_layers: number of lstm layers
        hidden_dim: dimensionality of the hidden layer of lstms
    Inputs: sequence of shape (batch_size, seq_len, in_dim)
    Output: sequence of shape (batch_size, seq_len, 1)
    """

    def __init__(self, in_dim, n_layers=1, hidden_dim=256):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.linear_hidden = nn.Sequential(nn.Linear(hidden_dim*self.n_layers, 1), nn.Sigmoid())

    def forward(self, input):
        batch_size, seq_len = input.size(0), input.size(1)
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")
        c_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")

        recurrent_features, (h_out,c_out) = self.lstm(input, (h_0, c_0))
        # outputs = self.linear_recurrent(recurrent_features.contiguous().view(batch_size*seq_len, self.hidden_dim))
        # outputs = outputs.view(batch_size, seq_len, 1)
        return self.linear_hidden(h_out.view(batch_size,self.hidden_dim*self.n_layers))

In [9]:
class LSTMDiscriminatorCO(nn.Module):
    """An LSTM based discriminator. It expects a sequence as input and outputs a probability for each element. 
    Args:
        in_dim: Input noise dimensionality
        n_layers: number of lstm layers
        hidden_dim: dimensionality of the hidden layer of lstms
    Inputs: sequence of shape (batch_size, seq_len, in_dim)
    Output: sequence of shape (batch_size, seq_len, 1)
    """

    def __init__(self, in_dim, n_layers=1, hidden_dim=256):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.linear_cellst = nn.Sequential(nn.Linear(hidden_dim*n_layers, 1), nn.Sigmoid())

    def forward(self, input):
        batch_size, seq_len = input.size(0), input.size(1)
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")
        c_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")

        recurrent_features, (h_out,c_out) = self.lstm(input, (h_0, c_0))
        return self.linear_cellst(c_out.view(batch_size,self.hidden_dim*self.n_layers))

In [10]:
class LSTMDiscriminatorRF(nn.Module):
    """An LSTM based discriminator. It expects a sequence as input and outputs a probability for each element. 
    Args:
        in_dim: Input noise dimensionality
        n_layers: number of lstm layers
        hidden_dim: dimensionality of the hidden layer of lstms
    Inputs: sequence of shape (batch_size, seq_len, in_dim)
    Output: sequence of shape (batch_size, seq_len, 1)
    """

    def __init__(self, in_dim,n_layers=1, hidden_dim=256):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layers, batch_first=True)
        self.linear_recurrent = nn.Sequential(nn.Linear(hidden_dim, 1), nn.Sigmoid())
        self.bilinear_out = nn.Bilinear(1,1,1)

    def forward(self, input):
        batch_size, seq_len = input.size(0), input.size(1)
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")
        c_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to("cuda:0")

        recurrent_features, (h_out,c_out) = self.lstm(input, (h_0, c_0))
        outputs = self.linear_recurrent(recurrent_features.contiguous().view(batch_size*seq_len, self.hidden_dim))
        outputs = outputs.view(batch_size, seq_len, 1)
        return outputs

In [11]:
batch_size = 16
input_dimenstions = 1
layers = 4
hidden_size = 1024
dataloader = DataLoader(dataset,batch_size=16,shuffle = True)
disc1 = LSTMDiscriminatorRF(input_dimenstions,n_layers=layers,hidden_dim=hidden_size).to("cuda:0")
disc2 = LSTMDiscriminatorHC(input_dimenstions,n_layers=layers,hidden_dim=hidden_size).to("cuda:0")
disc3 = LSTMDiscriminatorHO(input_dimenstions,n_layers=layers,hidden_dim=hidden_size).to("cuda:0")
disc4 = LSTMDiscriminatorCO(input_dimenstions,n_layers=layers,hidden_dim=hidden_size).to("cuda:0")
optimizer1 = optim.Adam(disc1.parameters(), lr=1e-4)
optimizer2 = optim.Adam(disc2.parameters(), lr=1e-4)
optimizer3 = optim.Adam(disc3.parameters(), lr=1e-4)
optimizer4 = optim.Adam(disc4.parameters(), lr=1e-4)
criterion = nn.BCELoss().to(dataset.device)


In [12]:
print("epoch |  bnum  | errD1  | errD2  | errD3  | errD4")
for epoch in range(30):
    for i, (data,label) in enumerate(dataloader, 0):
        # print(data.device,label.device)
        disc1.zero_grad()
        output = disc1(data)
        errD1 = criterion(output, label)
        errD1.backward()
        optimizer1.step()
        #---
        output = disc2(data)
        plabel = label[:,0,:]
        output = output.view(-1,1)
        errD2 = criterion(output, plabel)
        errD2.backward()
        optimizer2.step()
        #---
        output = disc3(data)
        plabel = label[:,0,:]
        output = output.view(-1,1)
        errD3 = criterion(output, plabel)
        errD3.backward()
        optimizer3.step()
        #---
        output = disc4(data)
        plabel = label[:,0,:]
        output = output.view(-1,1)
        errD4 = criterion(output, plabel)
        errD4.backward()
        optimizer4.step()
        
        print(f"{epoch:5} | {i:6} | {errD1.item():6.3} | {errD2.item():6.3} | {errD3.item():6.3} | {errD4.item():6.3}",end = "\r",flush=True)
        if i%100 == 0:
            print()
    print("\nepoch finished: ",epoch)

epoch |  bnum  | errD1  | errD2  | errD3  | errD4


ValueError: Using a target size (torch.Size([16, 32, 1])) that is different to the input size (torch.Size([16, 32, 19])) is deprecated. Please ensure they have the same size.

In [ ]:
c,label = next(iter(DataLoader(dataset=dataset,batch_size=len(dataset),shuffle=True)))
output = disc4(c)
plabel = label[:,0,:]
output = output.view(-1,1)
print(plabel.shape, output.shape)
corrects = []
for i,j in zip(plabel,output):
    print(i.item(),round(j.item()))
    corrects.append(int(i.item() == j.item()))


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.91 GiB (GPU 0; 8.00 GiB total capacity; 3.90 GiB already allocated; 2.90 GiB free; 4.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF